In [6]:
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)


# Parameters
Lx, Lz = 4, 1
Nx, Nz = 1024, 256
Rayleigh = 2e6
Prandtl = 1
dealias = 3/2
stop_sim_time = 50
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

# Bases
coords = d3.CartesianCoordinates('x', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
b = dist.Field(name='b', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
tau_p = dist.Field(name='tau_p')
tau_b1 = dist.Field(name='tau_b1', bases=xbasis)
tau_b2 = dist.Field(name='tau_b2', bases=xbasis)
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=xbasis)

# Substitutions
kappa = 2.23e-5
nu = 1.5e-5
x, z = dist.local_grids(xbasis, zbasis)
ex, ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)
grad_u = d3.grad(u) + ez*lift(tau_u1) # First-order reduction
grad_b = d3.grad(b) + ez*lift(tau_b1) # First-order reduction

# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, b, u, tau_p, tau_b1, tau_b2, tau_u1, tau_u2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p = 0")
problem.add_equation("dt(b) - kappa*div(grad_b) + lift(tau_b2) = - u@grad(b)")
problem.add_equation("dt(u) - nu*div(grad_u) + grad(p) - b*ez + lift(tau_u2) = - u@grad(u)")
problem.add_equation("b(z=0) = Lz")
problem.add_equation("u(z=0) = 0")
problem.add_equation("b(z=Lz) = 0")
problem.add_equation("u(z=Lz) = 0")
problem.add_equation("integ(p) = 0") # Pressure gauge

# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time

# Initial conditions
b.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
b['g'] *= z * (Lz - z) # Damp noise at walls
b['g'] += Lz - z # Add linear background

# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_task(b, name='buoyancy')
vorticity = -d3.div(d3.skew(u))
snapshots.add_task(vorticity, name='vorticity')

# CFL
CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=5, safety=0.5, threshold=0.05,
             max_change=1.5, min_change=0, max_dt=max_timestep)
CFL.add_velocity(u)

# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')

# Main loop
startup_iter = 10
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

2024-04-06 11:54:24,536 subsystems 0/1 INFO :: Building subproblem matrices 1/64 (~2%) Elapsed: 0s, Remaining: 5s, Rate: 1.3e+01/s
2024-04-06 11:54:24,739 subsystems 0/1 INFO :: Building subproblem matrices 7/64 (~11%) Elapsed: 0s, Remaining: 2s, Rate: 2.5e+01/s
2024-04-06 11:54:24,973 subsystems 0/1 INFO :: Building subproblem matrices 14/64 (~22%) Elapsed: 1s, Remaining: 2s, Rate: 2.7e+01/s
2024-04-06 11:54:25,211 subsystems 0/1 INFO :: Building subproblem matrices 21/64 (~33%) Elapsed: 1s, Remaining: 2s, Rate: 2.8e+01/s
2024-04-06 11:54:25,444 subsystems 0/1 INFO :: Building subproblem matrices 28/64 (~44%) Elapsed: 1s, Remaining: 1s, Rate: 2.8e+01/s
2024-04-06 11:54:25,682 subsystems 0/1 INFO :: Building subproblem matrices 35/64 (~55%) Elapsed: 1s, Remaining: 1s, Rate: 2.9e+01/s
2024-04-06 11:54:25,917 subsystems 0/1 INFO :: Building subproblem matrices 42/64 (~66%) Elapsed: 1s, Remaining: 1s, Rate: 2.9e+01/s
2024-04-06 11:54:26,149 subsystems 0/1 INFO :: Building subproblem matri

/opt/anaconda3/envs/dedalus3/lib/python3.12/site-packages/dedalus/core/transforms.py:162: SyntaxWarning: invalid escape sequence '\s'
  """
/opt/anaconda3/envs/dedalus3/lib/python3.12/site-packages/dedalus/core/transforms.py:334: SyntaxWarning: invalid escape sequence '\s'
  """
/opt/anaconda3/envs/dedalus3/lib/python3.12/site-packages/dedalus/core/transforms.py:633: SyntaxWarning: invalid escape sequence '\s'
  """


KeyboardInterrupt: 

In [7]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from dedalus.extras import plot_tools
import pathlib
from dedalus.tools import logging
from dedalus.tools import post
from dedalus.tools.parallel import Sync

# Function to plot snapshots
def plot_snapshots(filename, output_path):
    """Save plot of specified tasks for given range of analysis writes."""
    
    # Plot settings
    tasks = ['buoyancy', 'vorticity']
    scale = 1.5
    dpi = 200
    title_func = lambda sim_time: 't = {:.3f}'.format(sim_time)
    savename_func = lambda write: 'write_{:06}.png'.format(write)

    # Layout
    nrows, ncols = 2, 1
    image = plot_tools.Box(4, 1)
    pad = plot_tools.Frame(0.3, 0, 0, 0)
    margin = plot_tools.Frame(0.2, 0.1, 0, 0)

    # Create multifigure
    mfig = plot_tools.MultiFigure(nrows, ncols, image, pad, margin, scale)
    fig = mfig.figure

    # Ensure output directory exists
    pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

    # Plot writes
    with h5py.File(filename, mode='r') as file:
        frame_count = file['tasks'][tasks[0]].shape[0]
        for index in range(frame_count):
            for n, task in enumerate(tasks):
                # Build subfigure axes
                i, j = divmod(n, ncols)
                axes = mfig.add_axes(i, j, [0, 0, 1, 1])
                # Call 3D plotting helper, slicing in time
                dset = file['tasks'][task]
                plot_tools.plot_bot_3d(dset, 0, index, axes=axes, title=task, even_scale=True, visible_axes=False)
            # Add time title
            title = title_func(file['scales/sim_time'][index])
            title_height = 1 - 0.5 * mfig.margin.top / mfig.fig.y
            fig.suptitle(title, x=0.44, y=title_height, ha='left')
            # Save figure
            savename = savename_func(file['scales/write_number'][index])
            savepath = pathlib.Path(output_path).joinpath(savename)
            fig.savefig(str(savepath), dpi=dpi)
            fig.clear()
    plt.close(fig)

In [10]:
# Directory containing the HDF5 files
snapshot_dir = pathlib.Path('snapshots')

# Get a sorted list of HDF5 files
snapshot_files = sorted(snapshot_dir.glob('snapshots_s*.h5'))
print(snapshot_files)

# Output directory for the plots
output_directory = './frames'

# Loop over each file and process it
for file_path in snapshot_files:
    print(f"Processing {file_path.name}...")
    plot_snapshots(str(file_path), output_directory)

[PosixPath('snapshots/snapshots_s1.h5'), PosixPath('snapshots/snapshots_s2.h5'), PosixPath('snapshots/snapshots_s3.h5'), PosixPath('snapshots/snapshots_s4.h5')]
Processing snapshots_s1.h5...
Processing snapshots_s2.h5...
Processing snapshots_s3.h5...
Processing snapshots_s4.h5...


In [11]:
import cv2
import os
import numpy as np

# Directory containing the frames
frames_dir = 'frames'
# Output video file
output_video = 'output_video.avi'

# Assuming all images are the same size, get dimensions from the first image
sample_img = cv2.imread(os.path.join(frames_dir, os.listdir(frames_dir)[0]))
height, width, layers = sample_img.shape

# Video properties
frame_rate = 20  # Frames per second
frame_size = (width, height)

# Retrieve file names and sort them
frame_files = [f for f in os.listdir(frames_dir) if os.path.isfile(os.path.join(frames_dir, f))]
frame_files.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(output_video, fourcc, frame_rate, frame_size)

# Add images to video
for frame in frame_files:
    img_path = os.path.join(frames_dir, frame)
    img = cv2.imread(img_path)
    video_writer.write(img)

# Release the video writer
video_writer.release()
